# Overview: Analyzing TARDIS spectra using built-in visualization tools

How do we figure out which ion contributed to a specific spectral feature we are looking at in the spectrum? 

That is one of the most important questions we would like to answer from the observed spectra.

In TARDIS we can do this by tracking the Monte Carlo packet properties, specifically the last interaction before the packet escapes from the outermost shell of the ejecta. 

This notebook introduces built-in TARDIS visualization tools and guides you through how to use them to interpret the outputs. 
These tools can help identify line profiles, line-forming regions, line ratios, and ionization states, which provide insights into whether your simulation is physically consistent and where potential adjustments are needed.


# Run a TARDIS simulation

$\blacktriangleright$ Run the cell below to run a TARDIS simulation that has a uniform abundance. 

In [ ]:
from tardis import run_tardis
from tardis.io.configuration.config_reader import Configuration

config = Configuration.from_yaml("configs_and_data/tardis_example.yml")

sim = run_tardis(
    config,
    virtual_packet_logging=True,
    show_convergence_plots=True,
    export_convergence_plots=True,
    log_level="INFO",
)


# SDEC (Spectral element DEComposition) Plot


The SDEC plot in TARDIS visually presents what physical interactions contributed to the formation of the spectrum. The plot shows the photospheric blackbody SED and the emergent spectrum after the packets go through the ejecta, with packet contributions color-coded by the type of last interaction the energy packets experienced before they travel beyond the outermost shell of the ejecta. 

Let's first plot a SDEC plot and then see what's inside later.

🔧 **SDEC plot settings**

The common settings are:
- `packet_type`: real or virtual, indicates which type of packets to use<br>
        - **Real packets**: The Monte Carlo packets that started from the photosphere<br>
        - **Virtual packets**: The virtual packets are a technique to increase spectral resolution, see [documentation here](https://tardis-sn.github.io/tardis/physics/spectrum/virtualpackets.html#:~:text=10%5D.-,Virtual%20Packets,-%C2%B6)<br>
- `packet_wvl_range`: By default the SDEC spans the entire requested wavelength; you can use this setting to select a specific wavelength range
- `species_list`: Only color-code a specific set of elements and/or ions, such as `species_list = ['Si II', 'S I-V', 'Ca I-III']`. 


See the [SDEC tutorial](https://tardis-sn.github.io/tardis/analyzing_tardis/visualization/how_to_sdec_plot.html) for more optional settings.

--- 
## Plot a Static SDEC

$\blacktriangleright$ Run the cell below to produce a static matplotlib SDEC plot with default settings. 


In [ ]:
from tardis.visualization import SDECPlotter
from astropy import units as u

plotter = SDECPlotter.from_simulation(sim)

# generate a static matplotlib plot
plotter.generate_plot_mpl()


$\blacktriangleright$ Run the cell below, which specifies a specific wavelength range and ions.

In [ ]:
plotter.generate_plot_mpl(
    packets_mode="virtual",
    packet_wvl_range=[3000, 9000] * u.AA,
    species_list=["O I-III", "Mg I-III", "S I-III", "Si I-III", "Ca I-III", "Ar I-III"],
)


## What is the SDEC Plot?

Let's look at each of the components in this plot

- **<font color=red> Blackbody Photosphere </font>** (The red dashed line in the SDEC plot):<br>
  This shows the luminosity density distribution of the energy packets when they are first initialized at the photosphere, which is a blackbody distribution. 

- **<font color=DodgerBlue> Spectrum </font>** (The blue dashed line in the SDEC plot):<br>
  Depending on your choice of `packets_mode` (`real` or `virtual`), this shows the `spectrum_real_packets` or `spectrum_virtual_packets` from the spectrum_solver within the simulation object. The luminosity density at each wavelength is the cumulative luminosity of the escaped packets binned by frequency or wavelength, such as: 
  $$
  L_{\lambda} = \Sigma_1^N l_i / \Delta \lambda
  $$
  where $l_i$ is the luminosity ([erg/s]) of each packet, N is the count of the packets in wavelength bin, and $\Delta  \lambda$ is the size of the wavelength bin [$\AA$].

- **<span style="background-color: rgba(255, 255, 0, 0.3); padding: 4px;">Emitted Luminosity Density</span>** (The plotted spectral contribution with $L_{\lambda}$ > 0):<br>
  While the spectrum shows the total emitted luminosity density, the contributions within are color-coded by the last interaction type of the packet, which are:<br>
    - No interaction: The packet experienced no interaction from starting at the photosphere to the outer edge of the last shell. 
    - Electron Scatter: The packet experienced electron scattering as its last interaction before escaping <br>
      ($\blacktriangleright$<u>**Question 1**</u>: Does the packet change its lab frame energy and wavelength after this process?)
    - Line interaction: The packet experienced line interaction (bound-bound transition) as its last interaction before escaping <br>
      ($\blacktriangleright$<u>**Question 2**</u>: Does the packet change its co-moving frame frequency during line interaction?)

- **<span style="background-color: rgba(255, 255, 0, 0.3); padding: 4px;">Absorbed Luminosity Density</span>** (The plotted spectral contribution with $L_{\lambda}$ < 0):<br>
  <div style="background-color:rgb(95, 83, 77); border-left: 4px solid #f44336; padding: 10px; margin: 10px 0;">
  <strong>⚠️ Warning:</strong> These are NOT the "reabsorbed_packets" that traveled back inside the photosphere (which are discarded), rather these absorbed luminosity densities are plotting the cumulative luminosity of the packets **BEFORE** their last line interaction. 
  </div>
  Among the emitted packets, there are a group of packets that experienced line interaction as their last interaction. This process represents that the photons are absorbed and re-emitted at a (different) wavelength and angle. This **fluorescence** process redistributes the ion energy across the spectrum. The packet luminosity **AFTER** the line interaction is counted into the emitted luminosity, and the packet luminosity **BEFORE** the line interaction is regarded as absorptions, plotted as $-L_{\lambda}$.
---
<details>
<summary><strong>💡 <u>Click here for Hint for Question 1</u> </strong></summary>
YES, for any interaction event, TARDIS transform both energy and frequency of the packet into the co-moving frame, and a new direction is generated, then the packet energy and frequency is transformed back to the lab-frame (Only if the direction is the same, then the packet energy stays the same in the lab frame), see <a href="https://tardis-sn.github.io/tardis/physics/montecarlo/propagation.html#physical-interactions:~:text=employed%20by%20TARDIS.-,Performing%20an%20Interaction,-%C2%B6" target="_blank">documentation here</a>.
</details>

<details>
<summary><strong>💡 <u>Click here for Hint for Question 2</u> </strong></summary>
It depends on the <b>line_interaction_type</b> setting in the configuration (scatter, downbranch, macroatom). Recall from Day 3 or see <a href="https://tardis-sn.github.io/tardis/physics/montecarlo/lineinteraction.html" target="_blank">documentation here</a>. Only in the "scatter" line_interaction_type setting will the packet have the same co-moving frequency when interacting with resonant lines.
</details>


<font size=+3>&#9998;</font> Double click this cell to write your answer here.

## Plot an interactive SDEC

For exploration purposes, an interactive version of the SDEC plot is also available through `plotter.generate_plot_ply()`, in which you can zoom in on regions interactively. (Double click to zoom out to the full range.)

$\blacktriangleright$ Run the cell below to plot an interactive SDEC and explore different wavelength regions to answer the question below: 

$\blacktriangleright$<u>**Question 3**</u>: Based on the SDEC plot, identify which wavelength regions are dominated by continuum opacity (contributed by electron scattering), and which regions are dominated by line opacity (contributed by line interaction) in this simulation. 

---
<details>
<summary><strong>💡 <u>Hint for Question 3</u> </strong></summary>
In this specific simulation, the SDEC plot shows that line opacity dominates in the optical region, with strong contributions from ions like S II, Si II, and Ca II. Electron scattering is only significant in wavelength regions lacking strong lines. This reflects the strong wavelength dependence of opacity and highlights the limitations of the use of grey opacity in physical applications.
</details>

<font size=+3>&#9998;</font> Double click this cell to write your answer here.

In [ ]:
plotter.generate_plot_ply()

# Line Info Widget

The Line Info Widget provides an interactive way to explore atomic line data and identify which spectral lines contribute to specific wavelength regions using the last interaction information of the Monte Carlo packets. This widget is particularly useful for:

- **Line identification**: Finding which atomic transitions produce absorption/emission features at specific wavelengths
- **Ion analysis**: Understanding which ionization/excitation states of elements are present in different regions and show their fractions.

**Key Features**:
- **Interactive wavelength selection**: Click + hold to draw select box to filter the wavelength ranges to see contributing packets information
- **Element filtering**: Focus on specific elements or ions within selected wavelength
  - You can choose two different ways to filter the packets, by their emitted or absorbed wavelength, which corresponds to the wavelength before or after the last line interaction, (see the explanation for the SDEC plot above) 
- **Transition information**: View detailed atomic data for each line

*This widget complements the SDEC plot by providing the atomic physics context for spectral features.*

---
## Display the interactive widget
$\blacktriangleright$ Run the cell below to display the Line info widget

In [ ]:
from tardis.visualization import LineInfoWidget

line_info_widget = LineInfoWidget.from_simulation(sim)
line_info_widget.display()

## TASK: Si ionization ratio VS Temperature

$\blacktriangleright$ TASK: Use the Line Info Widget to investigate how ionization states change with luminosity:

1. Using the current simulation, select the wavelength range [1000, 7000] $\AA$ in the Line Info Widget and record the packet fractions for Si II and Si III.

2. Calculate the Si III/Si II ratio: This ratio indicates the ionization balance - higher ratios suggest more higher ionization.

3. Run another simulation with a modified config that has 60% of the original luminosity, and repeat Steps 1 and 2.

4. Compare the ionization ratio of these two simulations. Which one is higher and why? 

---
<details>
<summary><strong>💡 <u> Click here for Hint</u> </strong></summary>
Increasing the requested luminosity in TARDIS raises the inner boundary temperature, producing a hotter and more energetic radiation field. This enhances reaction rates, shifting the ionization balance toward higher stages. As a result, the Si III/Si II ratio increases with luminosity under otherwise identical conditions.
</details>

In [ ]:
### fill in value and run the cell

# from copy import deepcopy

# config_lower_luminosity = deepcopy(config)
# config_lower_luminosity.supernova.luminosity_requested = ???

# sim_lower_luminosity = run_tardis(
#     config_lower_luminosity,
#     virtual_packet_logging=True,
#     show_convergence_plots=True,
#     export_convergence_plots=True,
#     log_level="INFO",
# )


In [ ]:
### run this cell

# line_info_widget_lower_luminosity = LineInfoWidget.from_simulation(sim_lower_luminosity)
# line_info_widget_lower_luminosity.display()

In [ ]:
### fill in value and run the cell

# sim_Si_ionization_ratio = ???
# sim_lower_luminosity_Si_ionization_ratio = ???

# print(
#     f"The Si ionization ratio for the original simulation is {sim_Si_ionization_ratio:.2f}"
# )
# print(
#     f"The Si ionization ratio for the lower luminosity simulation is {sim_lower_luminosity_Si_ionization_ratio:.2f}"
# )


## Access line interacting ion fractions as a dataframe

Within the line information widget class, you can also directly access the aggregated packet fractional stats by `get_species_interactions`, which takes:
- `wavelength_range` as a required input 
- `filter_mode`: `packet_out_nu` (default) or `packet_in_nu`, which filters the packets based on their wavelength before or after the last line interaction.
--- 
$\blacktriangleright$ Run the cell below to check the packet fraction within the wavelength range of [1000, 7000] $\AA$, which should be dominated by S II.

In [ ]:
line_info_widget.get_species_interactions(
    wavelength_range=[1000, 7000], filter_mode="packet_out_nu"
)

# Last Interaction Velocity (LIV) Plot

Previously we looked at the SDEC plot and the Line Info Widget, both of which show the packet distribution grouped by packet wavelength.

How about the interaction location? For packets that experienced line interaction as their last interaction, where in the ejecta did the line interaction happen? 

This is essentially what's presented in the LIV plot, which offers information about the line-forming region for each ion. 

--- 
🔧 **LIV plot settings**

LIV plots share several similar settings as the SDEC plot, such as:
- `packet_type`: real or virtual, indicates which type of packets to use<br>
        - **Real packets**: The Monte Carlo packets that started from the photosphere<br>
        - **Virtual packets**: The virtual packets are a technique to increase spectral resolution, see [documentation here](https://tardis-sn.github.io/tardis/physics/spectrum/virtualpackets.html#:~:text=10%5D.-,Virtual%20Packets,-%C2%B6)<br>
- `packet_wvl_range`: By default the LIV plot includes all packets; you can use this setting to select packets that are within a specific wavelength range
- `species_list`: Only show a specific set of elements and/or ions, such as `species_list = ['Si II', 'S I-V', 'Ca']`. 


See the [LIV plot tutorial](https://tardis-sn.github.io/tardis/analyzing_tardis/visualization/how_to_liv_plot.html) for more optional settings.

---
## Plot a LIV plot using virtual VS real packets
$\blacktriangleright$ Run the cell below to see the LIV plot using "real" and "virtual" packets, respectively. 

In [ ]:
from tardis.visualization.tools.liv_plot import LIVPlotter
from matplotlib import pyplot as plt

LIVplotter = LIVPlotter.from_simulation(sim)
LIVplotter.generate_plot_mpl(
    packets_mode="real",
    xlog_scale=False,
    ylog_scale=True,
)
plt.title("LIV plot with real packets", fontsize=16)

LIVplotter.generate_plot_mpl(
    packets_mode="virtual",
    xlog_scale=False,
    ylog_scale=True,
)
plt.title("LIV plot with virtual packets", fontsize=16)


$\blacktriangleright$ **Questions** 
(After running the LIV plot above) 

- Question 1: What difference did you notice between the real vs virtual packets LIV plot and why is that?  
- Question 2: What determines the **quantity** of a line interaction at a given location physically, and what does the LIV plot tell us about ion opacity?

<details>
<summary><strong>💡 <u>Hint 1</u> </strong></summary>
The total number of packet counts is the major difference. Recall from Day 3, N virtual packets are spawned at the location where each interaction happens, hence the total number of virtual packets scales up with the number of interactions the real packets have.
</details>

<details>
<summary><strong>💡 <u>Hint 2</u> </strong></summary>
The quantity of a bound-bound interaction of an ion depends on the number of ions, the number of available photons and free electrons within the plasma that induce the interaction, which are related to the opacity of the ion. The LIV plot shows the ion opacity distribution, which is determined by a combination of the radiation field and ion number density.
</details>

--- 

<div style="background-color:rgb(95, 83, 77); border-left: 4px solid #f44336; padding: 10px; margin: 10px 0;">
<strong>Note:</strong> 
Related to the first question, some simulations might take a while to plot the LIV plot using virtual packets if there are MANY interactions that occurred. Try using real packets if needed in these cases.
</div>


<font size=+3>&#9998;</font> Double click this cell to write your answer here.

## TASK: Exploring Si II $\lambda\lambda$ 5972 vs 6355

LIV plots show the velocity distribution of bound-bound interactions of ions. Let's dive deeper into a specific case: the iconic Si II $\lambda\lambda$ 5972 and $\lambda\lambda$ 6355 doublet features in Type Ia supernovae. 

**Background**: These two features encode rich information about the structure of the SN Ia ejecta, and the relative feature strength of these two features is often correlated with the diversity within SNe Ia (Branch diagram). 

Understanding where these lines form in velocity space can help us interpret the physical conditions in the ejecta! 

---

$\blacktriangleright$**Your task**: 
1. Identify the unique identifier of the doublet Si II line transitions (`atomic_number`,`ion_number`,`level_number_lower`, `level_number_upper`) of the major spectral features ( $\lambda\lambda$ 5972, $\lambda\lambda$ 6355) using the atomic line list. 
2. Filter out the packets that interacted with these lines (select based on the unique identifier found in step 1 above) and map where these interactions occur in velocity space.
3. Overlay other information to discuss the physical conditions and potential reason behind the distribution you see (radiative temperature and the number density of the ion).
4. Estimate the Doppler velocity of the feature from the synthetic spectra of these two features and compare with the median velocity of the corresponding packets. 


This exercise will hopefully refresh some of the skills that you picked up from the previous days, such as accessing the atomic lines (from Days 2-3) and the last interaction packet information (from Day 3).

---
$\blacktriangleright$ Run the cell below to filter out the Si II line list

In [ ]:
TARGET_ATOMIC_NUMBER = 14  # Si
TARGET_ION_NUMBER = 1  # II

# Find line information for the features
line_list = (
    sim.plasma.atomic_data.lines.reset_index()
)  # Reset index to access index columns
ion_mask = (line_list["atomic_number"] == TARGET_ATOMIC_NUMBER) & (
    line_list["ion_number"] == TARGET_ION_NUMBER
)
Si_II_line_list = line_list[ion_mask]
Si_II_line_list

$\blacktriangleright$ Filter the line list based on wavelength, sort it by the `f_ul` column,
and find the strongest 2 lines contributing to each of the main features. 

- Si II 5972: try filtering the wavelength range [5950, 6200] $\AA$ 

- Si II 6355: try filtering the wavelength range [6340, 6380] $\AA$

In [ ]:
## filter the wavelength range
# subdf_line_list = Si_II_line_list.loc[]

## sort by the f_ul column
# subdf_line_list.sort_values("f_ul", ascending=False)

$\blacktriangleright$ Uncomment the cell below and fill in the missing parts. 

Access the last line interaction packet information, and filter out the packets that interacted with the above lines. Plot the distribution in velocity space.

Use astropy units for quantity unit conversion.

In [ ]:
# import matplotlib.pyplot as plt

# ### Define the target line IDs for the Si II doublet
# TARGET_LINE_LEVELS = [
#     [(14, 1,  ???,  ???), (14, 1,  ???,  ???)],
#     [(14, 1, ???,  ???), (14, 1, ???,  ???)],
# ]
# LABELS = ["Si II 5972", "Si II 6355"]

# ### Filter packets with line interaction as last interaction
# line_interaction_mask = ???

# ### Get interaction radius and convert to velocity
# interaction_location_in_radius = (
#     sim.transport.transport_state.last_interaction_in_r[line_interaction_mask] * u.cm
# )
# interaction_location_in_velocity =  ???

# ### Get the packet line interaction information
# line_list_all = sim.plasma.atomic_data.lines
# packet_line_ids = sim.transport.transport_state.last_line_interaction_in_id[
#     line_interaction_mask
# ]
# packet_line_info = line_list_all.iloc[packet_line_ids]


# ### Plot Si II doublet formation regions as a function of velocity
# fig, ax = plt.subplots(figsize=(12, 6))
# velocities = sim.simulation_state.v_inner.to(u.km / u.s).value

# for i, target_line_level in enumerate(TARGET_LINE_LEVELS):
#     # Find packets that interacted with these specific lines
#     target_mask = packet_line_info.index.isin(target_line_level)
#     interaction_velocities = interaction_location_in_velocity[target_mask].value
#     ax.hist(
#         interaction_velocities,
#         bins=velocities,
#         label=LABELS[i],
#         alpha=0.6,
#         linewidth=2,
#         histtype="stepfilled",
#     )

# ### Add temperature profile on secondary axis
# ax_temp = ax.twinx()
# ax_temp.plot(???, color= "r")
# ax_temp.set_ylabel("Temperature (K)", color="red")

# ### Add number density of each level as well
# ax_num_density = ax.twinx()
# levels = [?????]
# for level in levels:
#     ax_num_density.plot(
#         sim.simulation_state.v_inner.to(u.km / u.s).value,
#         sim.plasma.level_number_density.loc[TARGET_ATOMIC_NUMBER, TARGET_ION_NUMBER, level],
#         ls="--",
#         alpha=0.7,
#         label=f"Level {level}",
#     )
#     ax_num_density.spines["right"].set_position(("outward", 60))
#     ax_num_density.set_ylabel("Si II Number Density (cm$^{-3}$)")
# ax_num_density.legend(loc="upper left")

# #### Set labels and title
# ax.set_xlabel("Velocity (km/s)")
# ax.set_ylabel("Number of Line Interactions")
# ax.set_title("Si II Formation Regions")
# ax.legend(loc="upper right")
# ax.grid(True, alpha=0.3)


$\blacktriangleright$ Estimate the Doppler velocity of the two Si II features using the synthetic spectrum: 

**Velocity calculation from absorption minima:**
$$v = c \times \frac{\lambda_{rest} - \lambda_{obs}}{\lambda_{rest}}$$


And compare it to the median velocity of the packets that contribute to those features. 


In [ ]:
### Run this cell to get the median velocity of the interaction site
import numpy as np

for i, target_line_level in enumerate(TARGET_LINE_LEVELS):
    target_mask = packet_line_info.index.isin(target_line_level)
    interaction_velocities = interaction_location_in_velocity[target_mask].value
    print(
        f"label: {LABELS[i]}, median velocity: {np.median(interaction_velocities):.0f} km/s"
    )


In [ ]:
# Your code/analysis to estimate the feature Doppler shift by identifying the absorption minima location

---

$\blacktriangleright$ **Questions** 
(After finish the task above) 

- Question 1: Which feature has a faster expansion velocity measured from the spectrum? How about from the median velocity location of the interaction site? Do they match?
- Question 2: Quantitatively, does the velocity measured from the spectrum match with the median interaction site location?

<details>
<summary><strong>💡 <u> Click here for Hint 1</u> </strong></summary>
**In this simulation**, the Si II 6355 has a faster expansion velocity measured from the spectrum. This is also reflected in the median interaction site velocity of Si II 6355.
</details>

<details>
<summary><strong>💡 <u> Click here for Hint 2</u> </strong></summary>
These values are not expected to be the same. The Doppler shift velocity measured from the absorption minima of a spectral feature is a combined effect from interaction site velocity and the traveling direction of the photons, from all the contributing ions.
</details>

---

# **Summary**

Congratulations! We have gone through the following powerful visualization tools in TARDIS and some related exercises:

- **SDEC plots** - Understanding which ions contribute to spectral features  
- **LIV plots** - Mapping where line interactions occur in velocity space  
- **Line Info Widget** - Exploring atomic line data (when available)  

## **What's Next for today?**

**Notebook 2: Post-processing Hydro Models**
- Learn how to work with hydrodynamic simulation outputs using csvy files
- Optimize photospheric velocity using the dilution factor $W=0.5$ criterion  
- Apply visualization tools to analyze synthetic spectra from realistic explosion models

**Notebook 3: Exploring Parameterized Models** (for the afternoon)
- Systematically vary model parameters (luminosity, v_inner, time_explosion, density, abundance)
- Compare synthetic spectra with observations
- Understand parameter degeneracies and their physical effects

---

**Ready to continue?** Open `2_postprocessing_hydromodel.ipynb` to dive into realistic supernova modeling!

The visualization skills you've learned here will be essential for analyzing and interpreting the models in the upcoming notebooks.